In [6]:
from bs4 import BeautifulSoup
from glob import glob
from pathlib import Path
import json
from helpers.scrape_html import get_html2text_handler

In [15]:
def clean_webpage(page):
  page = page.replace("*", " ")
  page = page.replace("###", " ")
  page = page.replace("##", " ")
  page = page.replace("“", "\"")
  page = page.replace("”", "\"")
  page = page.encode("ascii", "ignore").decode()
  
  cleaned_page = []
  for line in page.split("\n"):
    line_text = line.strip()
    
    if not line_text:
      continue
    
    cleaned_line = " ".join(line_text.split())
    cleaned_page.append(cleaned_line)
  return " ".join(cleaned_page)
  

In [1]:
def get_question_answer(html_path):
  h = get_html2text_handler()
  
  html_path = Path(html_path)
  html_file = open(html_path)
  soup = BeautifulSoup(html_file)
  html_file.close()
  
  question = soup.find(id="kb_article_question")
  if not question:
    return None
  question = question.text.strip()
  
  answer = soup.find(id="kb_article_text")
  if not answer:
    return None
  answer = answer.prettify()
  
  return {
    "id": html_path.stem,
    "question": question,
    "answer": clean_webpage(h.handle(answer))
  } 

In [17]:
def extract_html_to_file():
  data = []
  
  for path in glob("./html-archive/*.html"):
    question_answer = get_question_answer(path)

    if not question_answer:
      continue
    
    data.append(question_answer)
  
  with open("./data/html-text.json", "w+") as html_data_file:
    json.dump({"data": data}, html_data_file)

In [18]:
extract_html_to_file()